In [1]:
import pandas as pd
from sklearn.metrics import f1_score
import torch
import torch.nn as nn

In [11]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [14]:
device

device(type='cuda', index=0)

In [2]:
train = pd.read_csv('/root/data/train_set.csv',sep='\t') 

In [3]:
text = train['text'].tolist()

def process_text(text, target_length=1057):
    numbers = list(map(int, text.split()))
    if len(numbers) < target_length:
        numbers.extend([0] * (target_length - len(numbers)))  # 填充0
    elif len(numbers) > target_length:
        numbers = numbers[:target_length]  # 截断
    return numbers

text_numbers = [process_text(item) for item in text]
text_tensor = torch.tensor(text_numbers, dtype=torch.long)

In [6]:
text_tensor[0]

tensor([2967, 6758,  339,  ..., 4741,  900, 1635])

In [16]:
vocab_size = 5000
embedding_dim = 128

In [17]:
embedding_layer = nn.Embedding(num_embeddings=vocab_size, embedding_dim=embedding_dim)

In [18]:
embedding_layer.to(device)

Embedding(5000, 128)

In [19]:
embedded_vectors = embedding_layer(text_tensor.to(device))

OutOfMemoryError: CUDA out of memory. Tried to allocate 100.80 GiB. GPU 0 has a total capacity of 31.50 GiB of which 28.08 GiB is free. Including non-PyTorch memory, this process has 0 bytes memory in use. Of the allocated memory 3.16 GiB is allocated by PyTorch, and 14.97 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
print("Embedded vectors shape:", embedded_vectors.shape)
print("Embedded vectors:", embedded_vectors)

In [ ]:
# 1. 使用TfidfVectorizer，转换成词向量
tfidfvector = TfidfVectorizer(max_features=3000)
train_vec = tfidfvector.fit_transform(train['text'])

In [ ]:
# 2. 模型训练 
## 2.1 转化成dataloader
class LabelDataset(Dataset):
    def __init__(self, label_sequences, multi_class_labels):
        self.label_sequences = [torch.tensor(seq, dtype=torch.long) for seq in label_sequences]
        self.multi_class_labels = multi_class_labels
 
    def __len__(self):
        return len(self.label_sequences)
 
    def __getitem__(self, idx):
        return self.label_sequences[idx], self.multi_class_labels[idx]

# 创建数据集和数据加载器
dataset = LabelDataset(train_vec, train['label'])
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

In [ ]:
## 2.2 转化成dataloader



In [ ]:
# 模型存储
joblib.dump(clf, 'tf_idf_ridge_classifier_model.joblib')

# 模型加载
# loaded_clf = joblib.load('tf_idf_ridge_classifier_model.joblib')

In [ ]:
# 3. 模型评估
envalue_pred = clf.predict(train_vec[:10000])
envalue_f1_score = f1_score(train['label'][:10000], envalue_pred, average='macro')
envalue_f1_score

In [ ]:
# 4. 模型预测
test = pd.read_csv('/root/data/test_a.csv', sep = '\t')
text_vec = tfidfvector.transform(test['text'])
pred = clf.predict(text_vec)

In [ ]:
pred

In [ ]:
# 5. 结果保存
test['label'] = pred
test['label'].to_csv("/root/data/TFIDF_Ridgeclassification.csv", index=False)  